# Hopfield Network: Associative Memory Model

## 1. Overview
A **Hopfield Network** is a recurrent neural network that serves as content-addressable ("associative") memory. It can store patterns and recall them from noisy or partial inputs using energy minimization.

## 2. Model Architecture
- **Neurons**: Binary units with states \( s_i \in \{-1, 1\} \) (or \( \{0, 1\} \))
- **Connections**: Symmetric weights \( w_{ij} = w_{ji} \) with no self-connections \( w_{ii} = 0 \)
- **Fully connected**: Every neuron connects to all others

## 3. Key Formulas

### Energy Function
The network's stability is defined by:
$$
E = -\frac{1}{2} \sum_{i=1}^N \sum_{j=1}^N w_{ij} s_i s_j
$$
- Stable states are local minima of \( E \)

### Weight Learning (Hebbian Rule)
For \( P \) stored patterns \( \{\xi^\mu\} \):
$$
w_{ij} = \frac{1}{N} \sum_{\mu=1}^P \xi_i^\mu \xi_j^\mu \quad \text{(for } i \neq j\text{)}
$$

### State Update
Asynchronous update rule:
$$
s_i \leftarrow \text{sgn}\left( \sum_{j=1}^N w_{ij} s_j \right)
$$

## 4. Algorithm Steps

1. **Storage Phase**:
   - Compute weights via Hebbian learning
   - Normalize by \( 1/N \) (number of neurons)

2. **Recall Phase**:
   - Initialize network with input pattern
   - Update neurons asynchronously until convergence
   - Output stable state (retrieved memory)

## 5. Python Implementation


In [2]:
import numpy as np

class HopfieldNetwork:
    def __init__(self, size):
        self.weights = np.zeros((size, size))
    
    def train(self, patterns):
        """Store patterns using Hebbian learning"""
        for p in patterns:
            p = np.reshape(p, (-1, 1))
            self.weights += np.dot(p, p.T)
        np.fill_diagonal(self.weights, 0)  # No self-connections
        self.weights /= len(patterns[0])  # Normalize
    
    def recall(self, pattern, max_steps=100):
        """Retrieve pattern from memory"""
        s = pattern.copy()
        for _ in range(max_steps):
            for i in range(len(s)):  # Asynchronous updates
                s[i] = np.sign(np.dot(self.weights[i], s))
        return s

# Example Usage
if __name__ == "__main__":
    # Store letters (flattened 3x3 patterns)
    patterns = [
        [1, -1, 1, -1, 1, -1, 1, -1, 1],  # Checkerboard
        [1, 1, 1, -1, 1, -1, -1, 1, -1]    # 'T' shape
    ]
    
    hn = HopfieldNetwork(size=9)
    hn.train(patterns)
    
    # Test noisy recall
    noisy_pattern = [-1, -1, 1, -1, 1, -1, -1, 1, -1]  # Corrupted 'T'
    recalled = hn.recall(noisy_pattern)
    print("Recalled pattern:\n", np.reshape(recalled, (3, 3)))

Recalled pattern:
 [[ 1.  1.  1.]
 [-1.  1. -1.]
 [-1.  1. -1.]]
